### Importing the nucleotide sequences I am using and adding them to a string container.

In [1]:
def read_file(input_name):
    sequence = ''
    with open(input_name) as input_file:
        input_file.readline()
        while True:
            seq = input_file.readline().strip()
            if len(seq) == 0:
                break
            sequence += seq
    return sequence

input_name = 'Forrest B100B10 BAC sequence Rhg4.txt'
forrest_sequence = read_file(input_name)
print(f'First 50 nucleotides of Forrest sequence: {forrest_sequence[:50]}')

First 50 nucleotides of Forrest sequence: CCCTCGCGAGTTGGTTCAGCTGCTGCCTGAGGCTGGACGACCTCGCGGAG


In [19]:
# def read_file(input_name):
#     sequence = ''
#     with open(input_name) as input_file:
#         input_file.readline()  # skip header
        
#         seq = input_file.readline().strip()
#         while seq:
#             sequence += seq
#             seq = input_file.readline().strip()
#     return sequence

# input_name = 'Forrest B100B10 BAC sequence Rhg4.txt'
# forrest_sequence = read_file(input_name)
# print(f'First 50 nucleotides of Forrest sequence: {forrest_sequence[:50]}')

First 50 nucleotides of Forrest sequence: CCCTCGCGAGTTGGTTCAGCTGCTGCCTGAGGCTGGACGACCTCGCGGAG


In [18]:
s = ''
if s:
    print('True')
else:
    print('False')

False


In [2]:
input_name = 'Forrest SHMT.txt'
shmt = read_file(input_name)
print(f'First 50 nucleotides of SHMT gene sequence: {shmt[:50]}')

First 50 nucleotides of SHMT gene sequence: CAATGGCACCAATGCCCAATGGGAGATTTAAGTCAAGCCCAACATCAACC


#### From the results of my thesis, I know the boundaries of the genes in the Rhg4 region of interest. Moving from one end of the region to the other, trying to locate the genes using Python.

In [7]:
# finding gene boundaries
def find_start_end(gene, variant):
    # using find to find the start index
    # start index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(gene)
    while index >= 0:
        indices.append((index+1, index+len(gene)))  # bp locations of genes reported by databases start from 1
        count += 1
        if count % 10:
            print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(gene, index+len(gene))
    return indices

# variant = forrest_sequence
# gene = shmt
variant = 'CCCTCGCAGTTGG'
gene = 'TCG'
boundary_indices = find_start_end(gene, variant)
print(f'boundary_indices: {boundary_indices}')

found at index 3, len(variant): 13
boundary_indices: [(4, 6)]


In [3]:
# finding gene boundaries
def find_start_end(gene, variant):
    # using find to find the start index
    # start index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(gene)
    while index >= 0:
        indices.append((index+1, index+len(gene)))  # bp locations of genes reported by databases start from 1
        count += 1
        if count % 10:
            print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(gene, index+len(gene))
    return indices

variant = forrest_sequence
gene = shmt
print(f'len(variant): {len(variant)}, len(gene): {len(gene)}')
# variant = 'CCCTCGCAGTTGG'
# gene = 'TCG'
boundary_indices = find_start_end(gene, variant)
print(f'boundary_indices: {boundary_indices}')

len(variant): 94336, len(gene): 5103
boundary_indices: []


In [4]:
# finding gene boundaries
def find_start_end(gene, variant):
    # using find to find the start index
    # start index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(gene)
    while index >= 0:
        indices.append((index+1, index+len(gene)))  # bp locations of genes reported by databases start from 1
        count += 1
        if count % 10:
            print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(gene, index+len(gene))
    return indices

variant = forrest_sequence
gene = shmt[2000:2005]
print(f'len(variant): {len(variant)}, len(gene): {len(gene)}')
# variant = 'CCCTCGCAGTTGG'
# gene = 'TCG'
boundary_indices = find_start_end(gene, variant)
print(f'boundary_indices: {boundary_indices}')

len(variant): 94336, len(gene): 1316
boundary_indices: []


### At this point, I ran into problem of Python not finding the gene sequence in Forrest sequence that we know should be there. I am taking the approach of comparing the Forrest sequence with the gene sequence using other bioinformatics tools.

#### I used BLAST (alignment tool) to zone in on the coding region of SHMT gene (while comparing sequences in BLAST). Adjusting the subsequence we're searching for should give us the result we're expecitng. The reason we don't is that the gene sequence we're searching for is oriented the wrong way.

In [4]:
def find_start_end(gene, variant):
    # using find to find the start index
    # start index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(gene)
    while index >= 0:
        indices.append((index+1, index+len(gene)))  # bp locations of genes reported by databases start from 1
        count += 1
        if count % 10:
            print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(gene, index+len(gene))
    return indices

variant = forrest_sequence
gene = shmt[2339:3657]
print(f'len(variant): {len(variant)}, len(gene): {len(gene)}')
# variant = 'CCCTCGCAGTTGG'
# gene = 'TCG'
boundary_indices = find_start_end(gene, variant)
print(f'boundary_indices: {boundary_indices}')

len(variant): 94336, len(gene): 1318
boundary_indices: []


#### The Gene sequence (Forrest SHMT coding sequence) we're searching for is oriented the wrong way in relation to the Variant sequence. Need to generate a reverse complement of the gene sequence before running the find boundaries function.

In [14]:
def complement(sequence):
    complement_seq = ""
    complement_dict = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    for letter in sequence:
        complement_seq += complement_dict[letter]
    return complement_seq

sequence = 'AAAACCCGGT'
complement = complement(sequence)
print(f'complement of {sequence}: {complement}')

complement of AAAACCCGGT: TTTTGGGCCA


In [26]:
def reverse_complement(sequence):
    complement_seq = ""
    complement_dict = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    for letter in sequence:
        complement_seq += complement_dict[letter]
    return complement_seq[::-1]

def complement(sequence):
    complement_seq = ""
    complement_dict = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    for letter in sequence:
        complement_seq += complement_dict[letter]
    return complement_seq

# -         21 987654321
#           0123456789 1
sequence = 'AGCTGCTGCCTG'
print(f'original sequence:   {sequence}')
complement_sequence = complement(sequence)
print(f'complement sequence: {complement_sequence}')
reverse_complement = reverse_complement(sequence)
print(f'reverse complement:  {reverse_complement}')

print(f'sequence left to right: {sequence[0:12]}')
print(f'sequence right to left: {sequence[-1:-13:-1]}')

print(f'middle 8 characters left to right: {sequence[2:10]}')
print(f'middle 8 characters right to left: {sequence[-3:-11:-1]}')

original sequence:   AGCTGCTGCCTG
complement sequence: TCGACGACGGAC
reverse complement:  CAGGCAGCAGCT
sequence left to right: AGCTGCTGCCTG
sequence right to left: GTCCGTCGTCGA
middle 8 characters left to right: CTGCTGCC
middle 8 characters right to left: CCGTCGTC


In [4]:
def reverse_complement(sequence):
    complement_seq = ""
    complement_dict = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    for letter in sequence:
        complement_seq += complement_dict[letter]
    return complement_seq[::-1]

def find_start_end(gene, variant):
    # using find to find the start index
    # start index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(gene)
    # print(f'index before entering while loop: {index}')
    while index >= 0:
        # print(f'entered while loop')
        indices.append((index+1, index+len(gene)+1))  # bp locations of genes reported by databases start from 1
        count += 1
        if count % 10:
            print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(gene, index + len(gene))
    return indices

sequence = shmt[2340:3657]
variant = forrest_sequence
gene = reverse_complement(sequence)
boundary_indices = find_start_end(gene, variant)
print(f'boundary_indices: {boundary_indices}')

found at index 1439, len(variant): 94336
boundary_indices: [(1440, 2757)]


#### Found the Forrest Serine hydroxymethyltransferase (SHMT) gene. It is where we expected it to be in the Forrest sequence.

### Performing the searches for start and for stop codons to confirm the location of the coding sequence of Forrest SHMT. We also know that the first index returned by the find_start_end function has to match up with the first index found by the find_start_codon function.

In [1]:
# why ATG as the codon?

def read_file(input_name):
    sequence = ''
    with open(input_name) as input_file:
        input_file.readline()
        while True:
            seq = input_file.readline().strip()
            if len(seq) == 0:
                break
            sequence += seq
    return sequence

# finding start codon
def find_start_codon(codon, variant, offset):
    # using find to find the codon index
    # codon index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(codon)
    while index >= 0:
        indices.append(index+offset+1)  # bp locations of genes reported by databases codon from 1
        count += 1
        # if count % 10:
        #     print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(codon, index+len(codon))
    return indices

input_name = 'Forrest B100B10 BAC sequence Rhg4.txt'
codon = 'ATG'
variant = read_file(input_name)
offset = 1439
ATG_codon_indices = find_start_codon(codon, variant, offset)
print(f'length of ATG codon indices: {len(ATG_codon_indices)}, ATG_codon_indices[:10]: {ATG_codon_indices[:10]}')

length of ATG codon indices: 1882, ATG_codon_indices[:10]: [1551, 1583, 1710, 1790, 1865, 1927, 1992, 2027, 2131, 2157]


In [1]:
# other strand of DNA
def read_file(input_name):
    sequence = ''
    with open(input_name) as input_file:
        input_file.readline()
        while True:
            seq = input_file.readline().strip()
            if len(seq) == 0:
                break
            sequence += seq
    return sequence

# finding start codon
def find_start_codon(codon, variant, offset):
    # using find to find the codon index
    # codon index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(codon)
    while index >= 0:
        indices.append(index+offset+1)  # bp locations of genes reported by databases codon from 1
        count += 1
        # if count % 10:
        #     print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(codon, index+len(codon))
    return indices

input_name = 'Forrest B100B10 BAC sequence Rhg4.txt'
codon = 'TAG'
variant = read_file(input_name)
offset = 1439
TAG_codon_indices = find_start_codon(codon, variant, offset)
print(f'length of ATG codon indices: {len(TAG_codon_indices)}, ATG_codon_indices[:10]: {TAG_codon_indices[:10]}')

length of ATG codon indices: 1236, ATG_codon_indices[:10]: [1725, 1904, 2023, 2281, 2293, 2482, 2619, 2701, 2818, 2824]


##### So far we're not finding the ATG start codon where it should be, but several things could be at play here. I am using both Python and bioinformatics tools to investigate.

#### Need for complement

In [7]:
def read_file(input_name):
    sequence = ''
    with open(input_name) as input_file:
        input_file.readline()
        while True:
            seq = input_file.readline().strip()
            if len(seq) == 0:
                break
            sequence += seq
    return sequence

def reverse_complement(sequence):
    complement_seq = ""
    complement_dict = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    for letter in sequence:
        complement_seq += complement_dict[letter.upper()]
    return complement_seq[::-1]

# finding start codon
def find_start_codon(codon, variant, offset):
    # using find to find the codon index
    # codon index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(codon)
    while index >= 0:
        indices.append(offset+index+1)  # bp locations of genes reported by databases codon from 1
        count += 1
        # if count % 10:
        #     print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(codon, index+len(codon))
    return indices

input_name = 'Forrest B100B10 BAC sequence Rhg4.txt'
codon = reverse_complement('ATG')
print(f'codon: {codon}')
variant = read_file(input_name)
offset = 1439
start_codon_indices = find_start_codon(codon, variant[offset:2757+1], offset)
print(f'length of TAC codon indices: {len(start_codon_indices)}')
print(f'start_codon_indices[-1:-11:-1]: {start_codon_indices[-1:11:-1]}')

codon: CAT
length of TAC codon indices: 12
start_codon_indices[-1:-11:-1]: []


In [1]:
def read_file(input_name):
    sequence = ''
    with open(input_name) as input_file:
        input_file.readline()
        while True:
            seq = input_file.readline().strip()
            if len(seq) == 0:
                break
            sequence += seq
    return sequence

def reverse_complement(sequence):
    complement_seq = ""
    complement_dict = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    for letter in sequence:
        complement_seq += complement_dict[letter.upper()]
    return complement_seq[::-1]

# finding start codon
def find_start_codon(codon, variant, offset):
    # using find to find the codon index
    # codon index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(codon)
    while index >= 0:
        indices.append(offset+index+1)  # bp locations of genes reported by databases codon from 1
        count += 1
        # if count % 10:
        #     print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(codon, index+len(codon))
    return indices

input_name = 'Forrest B100B10 BAC sequence Rhg4.txt'
codon = reverse_complement('ATG')
print(f'codon: {codon}')
variant = read_file(input_name)
offset = 1440
start_codon_indices = find_start_codon(codon, variant[offset:2757+1], offset)
print(f'length of TAC codon indices: {len(start_codon_indices)}')
print(f'TAC codon indices: {start_codon_indices}')
# print(f'start_codon_indices[-1:-11:-1]: {start_codon_indices[-1:11:-1]}')

codon: CAT
length of TAC codon indices: 12
TAC codon indices: [1467, 1636, 1768, 1938, 1999, 2110, 2383, 2541, 2569, 2666, 2697, 2755]


In [14]:
# accounting for the beginning of the codon being 2 position beyond the index returned

def read_file(input_name):
    sequence = ''
    with open(input_name) as input_file:
        input_file.readline()
        while True:
            seq = input_file.readline().strip()
            if len(seq) == 0:
                break
            sequence += seq
    return sequence

def reverse_complement(sequence):
    complement_seq = ""
    complement_dict = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    for letter in sequence:
        complement_seq += complement_dict[letter.upper()]
    return complement_seq[::-1]

# finding start codon
def find_start_codon(codon, variant, offset):
    # using find to find the codon index
    # codon index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(codon)
    while index >= 0:
        indices.append(2+offset+index+1)  # databases count from 1, adding 2 for the actual start of the codon
        count += 1
        # if count % 10:
        #     print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(codon, index+len(codon))
    return indices

input_name = 'Forrest B100B10 BAC sequence Rhg4.txt'
codon = reverse_complement('ATG')
print(f'codon: {codon[::-1]}')
variant = read_file(input_name)
offset = 1440
start_codon_indices = find_start_codon(codon, variant[offset:2757+1], offset)
print(f'TAC codon indices: {start_codon_indices}')

codon: TAC
TAC codon indices: [1469, 1638, 1770, 1940, 2001, 2112, 2385, 2543, 2571, 2668, 2699, 2757]


In [ ]:
# finding the gene's stop codon recursively - not much use: not recursive by nature

def find_stop_codon_iter(codons, variant, offset):
    indices = []
    for codon in codons:
        index = find_stop_codon_recurisve(codon, variant, offset)
        if index >= 0:
            indices.append(index+1)  # plus 1 because biological databases start count from 1, not 0
    return indices

def find_stop_codon_recursive(codon, variant, offset):
    # breaking recursion: if indices for stop codon is -1
    index = variant.find(codon, offset)
    if index == -1:
        return index

codons = ['TAA', 'TAG', 'TGA']
variant = forrest_sequence
offset = 1440

In [12]:
# finding the gene's stop codon - iterative algorithm: finds stop codon where indeed it is

def read_file(input_name):
    sequence = ''
    with open(input_name) as input_file:
        input_file.readline()
        while True:
            seq = input_file.readline().strip()
            if len(seq) == 0:
                break
            sequence += seq
    return sequence

def find_stop_codon(codons, variant, offset):
    indices = []
    for codon in codons:
        index = variant.find(codon, offset)
        while index >= 0:
            indices.append(index+1)  # plus 1 because biological databases start count from 1, not 0
            index = variant.find(codon, index+len(codon))
    return indices

input_name = 'Forrest B100B10 BAC sequence Rhg4.txt'
codons = ['TAA', 'TAG', 'TGA']
variant = read_file(input_name)
offset = 1439
stop_indices = find_stop_codon(codons, variant, offset)
print(f'indices of stop codons[:10]: {stop_indices[:10]}')
print(f'length of stop_indices: {len(stop_indices)}')
print(f'stop codon at 1445: {variant[1444:1441:-1]}')

indices of stop codons[:10]: [1445, 1471, 1482, 1551, 1557, 1642, 1803, 1824, 1920, 1940]
length of stop_indices: 6009
stop codon at 1445: TGA


### Found gene and its start and stop codons where they are. Do the sequences from the susceptible cultivars have the Forrest SHMT gene?

In [4]:
def read_file(input_name):
    sequence = ''
    with open(input_name) as input_file:
        input_file.readline()
        while True:
            seq = input_file.readline().strip()
            if len(seq) == 0:
                break
            sequence += seq
    return sequence

input_name = 'Williams 82 BAC 56G2 sequence.txt'
williams_sequence = read_file(input_name)
print(f'First 50 nucleotides of Williams sequence: {williams_sequence[:50]}')

First 50 nucleotides of Williams sequence: AAGCTTGCATGCTTTCTGGTTTCCTTGCCAATCACTGCATGGAACAATCT


In [19]:
def find_start_end(gene, variant):
    # using find to find the start index
    # start index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(gene)
    # print(f'index before entering while loop: {index}')
    while index >= 0:
        # print(f'entered while loop')
        indices.append((index+1, index+len(gene)+1))  # bp locations of genes reported by databases start from 1
        count += 1
        if count % 10:
            print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(gene, index + len(gene))
    return indices

gene = shmt
variant = williams_sequence
gene_boundaries = find_start_end(gene, variant)
print(f'boundaries of Forrest SHMT gene in Williams sequence: {gene_boundaries}')

boundaries of Forrest SHMT gene in Williams sequence: []


In [9]:
def find_start_end(gene, variant):
    # using find to find the start index
    # start index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(gene)
    # print(f'index before entering while loop: {index}')
    while index >= 0:
        # print(f'entered while loop')
        indices.append((index+1, index+len(gene)+1))  # bp locations of genes reported by databases start from 1
        count += 1
        if count % 10:
            print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(gene, index + len(gene))
    return indices

gene = shmt[2340:3657]
variant = williams_sequence
gene_boundaries = find_start_end(gene, variant)
print(f'boundaries of Forrest SHMT gene in Williams sequence: {gene_boundaries}')

boundaries of Forrest SHMT gene in Williams sequence: []


In [20]:
variant.find(gene)

-1

In [6]:
def Hamming_distance(p, q):
    mismatch_count = 0
    length = min(len(p), len(q))
    for i in range(length):
        if p[i] != q[i]:
            mismatch_count += 1
    return mismatch_count

p = shmt[2339:3658]
q = williams_sequence[91647:92966]  # indices based on BLAST
print(f'Number of mismatches: {Hamming_distance(p, q)}')

Number of mismatches: 2


In [8]:
def read_file(input_name):
    sequence = ''
    with open(input_name) as input_file:
        input_file.readline()
        while True:
            seq = input_file.readline().strip()
            if len(seq) == 0:
                break
            sequence += seq
    return sequence

input_name = 'Asgrow sequence.txt'
asgrow_sequence = read_file(input_name)
p = shmt[2339:3658]
q = asgrow_sequence[187599:188917]  # indices based on BLAST
print(f'Number of mismatches between coding sequence and Asgrow sequence: {Hamming_distance(p, q)}')

Number of mismatches between coding sequence and Asgrow sequence: 2


### At this point, we come to a closed loop: I am using a function that finds boundary indices of a gene, but for susceptible variants I need the indices to get accurate mismatch count. Solution: stop the search for Forrrest SHMT gene and build a phylogenetic tree.

In [16]:
def create_subsequence_from_file(input_name, output_name, BLAST_start, BLAST_end):
    input_file = open(input_name)
    output_file = open(output_name, 'w')
    input_seq = ''
    input_file.readline()
    while True:
        seq = input_file.readline().strip()
        if len(seq) == 0:
            break
        input_seq += seq
    
    output_seq = input_seq[BLAST_start:BLAST_end]
    output_file.write(output_seq)
    
    input_file.close()
    output_file.close()
    print(f'wrote output file {output_name}')

input_name = 'Williams 82 BAC 56G2 sequence.txt'
output_name = 'Williams SHMT-like - phylogeny.txt'
BLAST_start = 91648
BLAST_end = 92965
create_subsequence_from_file(input_name, output_name, BLAST_start, BLAST_end)

wrote output file Williams SHMT-like - phylogeny.txt


In [15]:
def create_subsequence_from_file(input_name, output_name, BLAST_start, BLAST_end):
    input_file = open(input_name)
    output_file = open(output_name, 'w')
    input_seq = ''
    input_file.readline()
    while True:
        seq = input_file.readline().strip()
        if len(seq) == 0:
            break
        input_seq += seq
    
    output_seq = input_seq[BLAST_start:BLAST_end]
    output_file.write(output_seq)
    
    input_file.close()
    output_file.close()
    print(f'wrote output file {output_name}')

input_name = 'Asgrow sequence.txt'
output_name = 'Asgrow SHMT-like - phylogeny.txt'
BLAST_start = 187600
BLAST_end = 188917
create_subsequence_from_file(input_name, output_name, BLAST_start, BLAST_end)

wrote output file Asgrow SHMT-like - phylogeny.txt


In [14]:
def create_subsequence_from_file(input_name, output_name, BLAST_start, BLAST_end):
    input_file = open(input_name)
    output_file = open(output_name, 'w')
    input_seq = ''
    input_file.readline()
    while True:
        seq = input_file.readline().strip()
        if len(seq) == 0:
            break
        input_seq += seq
    
    output_seq = input_seq[BLAST_start:BLAST_end:-1]
    output_file.write(output_seq)
    
    input_file.close()
    output_file.close()
    print(f'wrote output file {output_name}')

input_name = 'Forrest B100B10 BAC sequence Rhg4.txt'
output_name = 'Forrest SHMT - phylogeny.txt'
BLAST_start = 2757
BLAST_end = 1440
create_subsequence_from_file(input_name, output_name, BLAST_start, BLAST_end)

wrote output file Forrest SHMT - phylogeny.txt


In [20]:
def create_subsequence_from_file(input_name, output_name, BLAST_start, BLAST_end):
    input_file = open(input_name)
    output_file = open(output_name, 'w')
    input_seq = ''
    input_file.readline()
    while True:
        seq = input_file.readline().strip()
        if len(seq) == 0:
            break
        input_seq += seq
    
    output_seq = input_seq[BLAST_start:BLAST_end]
    output_file.write(output_seq)
    
    input_file.close()
    output_file.close()
    print(f'wrote output file {output_name}')

input_name = 'Glycine max SHMT mRNA.txt'
output_name = input_name[:-4] + ' - phylogeny.txt'
BLAST_start = 91
BLAST_end = 1264
create_subsequence_from_file(input_name, output_name, BLAST_start, BLAST_end)

wrote output file Glycine max SHMT mRNA - phylogeny.txt


In [21]:
def create_subsequence_from_file(input_name, output_name, BLAST_start, BLAST_end):
    input_file = open(input_name)
    output_file = open(output_name, 'w')
    input_seq = ''
    input_file.readline()
    while True:
        seq = input_file.readline().strip()
        if len(seq) == 0:
            break
        input_seq += seq
    
    output_seq = input_seq[BLAST_start:BLAST_end]
    output_file.write(output_seq)
    
    input_file.close()
    output_file.close()
    print(f'wrote output file {output_name}')

input_name = 'Essex SHMT.txt'
output_name = input_name[:-4] + ' - phylogeny.txt'
BLAST_start = 2341
BLAST_end = 2658
create_subsequence_from_file(input_name, output_name, BLAST_start, BLAST_end)

wrote output file Essex SHMT - phylogeny.txt


### Creating phylogenetic trees gives us interesting information, but doesn't get us closer to our goal. I went back to my thesis results and am back to finding the genes thought be unknown or predicted in 2015 in susceptible cultivars.

In [1]:
def read_file(input_name):
    sequence = ''
    with open(input_name) as input_file:
        input_file.readline()
        while True:
            seq = input_file.readline().strip()
            if len(seq) == 0:
                break
            sequence += seq
    return sequence

input_name = 'Forrest B100B10 BAC sequence Rhg4.txt'
forrest_sequence = read_file(input_name)
print(f'First 50 nucleotides of Forrest sequence: {forrest_sequence[:50]}')

First 50 nucleotides of Forrest sequence: CCCTCGCGAGTTGGTTCAGCTGCTGCCTGAGGCTGGACGACCTCGCGGAG


In [11]:
input_name = 'HST Factor 15 Transcript variant X2 mRNA.txt'
hst_factor15_var2_mRNA = read_file(input_name)
print(f'First 50 nucleotides of hst_factor15_var2_mRNA sequence: {hst_factor15_var2_mRNA[:50]}')
print(f'length: {len(hst_factor15_var2_mRNA)}')

First 50 nucleotides of hst_factor15_var2_mRNA sequence: AGTTGCAAATTCCAGGTTTTCTTCTTTTACTCAATTCTCTCTCTCTCTCT
length: 2416


In [16]:
def read_file(input_name):
    sequence = ''
    with open(input_name) as input_file:
        input_file.readline()
        while True:
            seq = input_file.readline().strip()
            if len(seq) == 0:
                break
            sequence += seq
    return sequence

def reverse_complement(sequence):
    complement_seq = ""
    complement_dict = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    for letter in sequence:
        complement_seq += complement_dict[letter.upper()]
    return complement_seq[::-1]

input_name = 'HST Factor 15 Transcript variant X2 mRNA.txt'
hst_factor15_var2_mRNA = read_file(input_name)
print(f'Forrest sequence 25099-25110:           {forrest_sequence[25099:25109]}')
print(f'hst_factor15_var2_mRNA 363-353:         {hst_factor15_var2_mRNA[363:353:-1]}')
rev_hst_factor15_var2_mRNA = reverse_complement(hst_factor15_var2_mRNA)
print(f'reverse hst_factor15_mRNA 363-353:      {rev_hst_factor15_var2_mRNA[353:363]}')

Forrest sequence 25099-25110:           ATCCAATTAA
hst_factor15_var2_mRNA 363-353:         TTAGGTTAAT
reverse hst_factor15_mRNA 363-353:      GATATTGCCT


In [13]:
subseq = rev_hst_factor15_var2_mRNA[353:363]
index = hst_factor15_var2_mRNA.find(subseq)
print(f'first 10 of mRNA: {hst_factor15_var2_mRNA[0:10]}')
print(f'subseq of mRNA:   {subseq}')
print(f'index: {index}')
print(f'mRNA 353-363: {hst_factor15_var2_mRNA[353:363]}')  # BLAST confirms this as aligning with Forrest genomic sequence

first 10 of mRNA: AGTTGCAAAT
subseq of mRNA:   GATATTGCCT
index: -1
mRNA 353-363: TTAATTGGAT


In [14]:
def reverse_complement(sequence):
    complement_seq = ""
    complement_dict = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    for letter in sequence:
        complement_seq += complement_dict[letter.upper()]
    return complement_seq[::-1]

subsequence = reverse_complement(hst_factor15_var2_mRNA[353:363])
print(f'first 10 of the mRNA: {hst_factor15_var2_mRNA[353:363]}')
print(f'reverse complement:    {subsequence}')

first 10 of the mRNA: TTAATTGGAT
reverse complement:    ATCCAATTAA


In [15]:
def reverse_complement(sequence):
    complement_seq = ""
    complement_dict = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    for letter in sequence:
        complement_seq += complement_dict[letter.upper()]
    return complement_seq[::-1]

def find_start_end(gene, variant):
    # using find to find the start index
    # start index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(gene)
    # print(f'index before entering while loop: {index}')
    while index >= 0:
        # print(f'entered while loop')
        indices.append((index+1, index+len(gene)+1))  # bp locations of genes reported by databases start from 1
        count += 1
        if count % 10:
            print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(gene, index + len(gene))
    return indices

dna_seq = hst_factor15_var2_mRNA[13:363]
gene = reverse_complement(dna_seq)
variant = forrest_sequence
boundary_indices = find_start_end(gene, variant)
print(f'boundary indices: {boundary_indices}')

found at index 25099, len(variant): 94336
boundary indices: [(25100, 25450)]


In [9]:
def reverse_complement(sequence):
    complement_seq = ""
    complement_dict = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    for letter in sequence:
        complement_seq += complement_dict[letter.upper()]
    return complement_seq[::-1]

def find_start_codon(codon, variant, offset):
    # using find to find the codon index
    # codon index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(codon)
    while index >= 0:
        indices.append(2+offset+index+1)  # databases count from 1, adding 2 for the actual start of the codon
        count += 1
        # if count % 10:
        #     print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(codon, index+len(codon))
    return indices

def find_stop_codon(codons, variant, offset):
    indices = []
    for codon in codons:
        index = variant.find(codon, offset)
        while index >= 0:
            indices.append(index+1)  # plus 1 because biological databases start count from 1, not 0
            index = variant.find(codon, index+len(codon))
    return indices

offset = 25099
start_codon = 'ATG'
rev_start_codon = reverse_complement(start_codon)
stop_codons = ['TAA', 'TAG', 'TGA']
variant = forrest_sequence
start_codon_indices = find_start_codon(rev_start_codon, variant[offset:25450+1], offset)
stop_codon_indices = find_stop_codon(stop_codons, variant, offset)
print(f'indices of start codon: {start_codon_indices}')
print(f'number of stop codons: {len(stop_codon_indices)}, first 10 indices of stop codon: {stop_codon_indices[:10]}')

indices of start codon: [25128, 25311, 25321, 25381]
number of stop codons: 4501, first 10 indices of stop codon: [25107, 25434, 25588, 25593, 25597, 25613, 25629, 25644, 25650, 25658]


#### Found the X2 transcript variant of HST Factor 15 mRNA and it indeed contains start and stop codons as we would expect an mRNA to have. The codon indices and gene boundaries returned by Python algorithm do not match up with my thesis findings. I am testing my theories by re-running these function with HST Factor 15 mRNA (no transcript variant).

In [23]:
def read_file(input_name):
    sequence = ''
    with open(input_name) as input_file:
        input_file.readline()
        while True:
            seq = input_file.readline().strip()
            if len(seq) == 0:
                break
            sequence += seq
    return sequence

input_name = 'Glycine max HST Factor 15 mRNA.txt'
hst_factor15_mRNA = read_file(input_name)
print(f'HST Factor 15 mRNA first 10 nucleotides: {hst_factor15_mRNA[:10]}')

HST Factor 15 mRNA first 10 nucleotides: CTGCAGCAGG


In [24]:
def reverse_complement(sequence):
    complement_seq = ""
    complement_dict = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    for letter in sequence:
        complement_seq += complement_dict[letter.upper()]
    return complement_seq[::-1]

def find_start_end(gene, variant):
    # using find to find the start index
    # start index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(gene)
    # print(f'index before entering while loop: {index}')
    while index >= 0:
        # print(f'entered while loop')
        indices.append((index+1, index+len(gene)+1))  # bp locations of genes reported by databases start from 1
        count += 1
        if count % 10:
            print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(gene, index + len(gene))
    return indices

gene = reverse_complement(hst_factor15_mRNA)
variant = forrest_sequence
boundaries = find_start_end(gene, variant)
print(f'indices: {boundaries}')

indices: []


In [26]:
def reverse_complement(sequence):
    complement_seq = ""
    complement_dict = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    for letter in sequence:
        complement_seq += complement_dict[letter.upper()]
    return complement_seq[::-1]

def find_start_codon(codon, variant, offset):
    # using find to find the codon index
    # codon index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(codon)
    while index >= 0:
        indices.append(2+offset+index+1)  # databases count from 1, adding 2 for the actual start of the codon
        count += 1
        # if count % 10:
        #     print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(codon, index+len(codon))
    return indices

def find_stop_codon(codons, variant, offset):
    indices = []
    for codon in codons:
        index = variant.find(codon, offset)
        while index >= 0:
            indices.append(index+1)  # plus 1 because biological databases start count from 1, not 0
            index = variant.find(codon, index+len(codon))
    return indices

offset = 20648
start_codon = 'ATG'
rev_start_codon = reverse_complement(start_codon)
stop_codons = ['TAA', 'TAG', 'TGA']
variant = forrest_sequence
start_codon_indices = find_start_codon(rev_start_codon, variant[offset:25433+1], offset)
stop_codon_indices = find_stop_codon(stop_codons, variant, offset)
print(f'indices of start codon: {start_codon_indices}')
print(f'number of stop codons: {len(stop_codon_indices)}, first 10 indices of stop codon: {stop_codon_indices[:10]}')

indices of start codon: [20661, 20711, 20727, 20745, 20822, 20831, 20854, 20914, 20930, 20941, 20975, 21003, 21029, 21056, 21059, 21095, 21143, 21146, 21233, 21272, 21481, 21489, 21614, 21620, 21654, 21716, 21729, 21828, 21926, 22157, 22160, 22174, 22357, 22520, 22610, 22662, 22690, 22740, 22752, 22805, 22822, 22885, 22943, 23037, 23223, 23232, 23354, 23407, 23434, 23645, 23676, 23732, 23807, 23816, 23874, 23905, 23909, 23936, 24009, 24035, 24046, 24049, 24066, 24118, 24171, 24191, 24234, 24420, 24449, 24461, 24511, 24514, 24518, 24571, 24591, 24696, 24739, 24852, 24911, 24921, 24928, 24931, 24934, 24937, 24941, 24944, 24950, 24957, 25028, 25055, 25078, 25128, 25311, 25321, 25381]
number of stop codons: 4766, first 10 indices of stop codon: [20666, 20676, 20745, 20813, 20817, 20848, 20873, 20943, 20954, 21009]


In [4]:
def read_file(input_name):
    sequence = ''
    with open(input_name) as input_file:
        input_file.readline()
        while True:
            seq = input_file.readline().strip()
            if len(seq) == 0:
                break
            sequence += seq
    return sequence

def reverse_complement(sequence):
    complement_seq = ""
    complement_dict = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    for letter in sequence:
        complement_seq += complement_dict[letter]
    return complement_seq[::-1]

def find_start_end(gene, variant):
    # using find to find the start index
    # start index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(gene)
    # print(f'index before entering while loop: {index}')
    while index >= 0:
        # print(f'entered while loop')
        indices.append((index+1, index+len(gene)+1))  # bp locations of genes reported by databases start from 1
        count += 1
        if count % 10:
            print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(gene, index + len(gene))
    return indices

input_name1 = 'Forrest B100B10 BAC sequence Rhg4.txt'
input_name2 = 'Forrest SHMT.txt'
forrest_sequence = read_file(input_name1)
print(f'First 50 nucleotides of Forrest sequence: {forrest_sequence[:50]}')
shmt = read_file(input_name2)
print(f'First 50 nucleotides of SHMT gene sequence: {shmt[:50]}')
sequence = shmt[2340:3657]
variant = forrest_sequence
gene = reverse_complement(sequence)
boundary_indices = find_start_end(gene, variant)
print(f'boundary_indices: {boundary_indices}')

found at index 1439, len(variant): 94336
boundary_indices: [(1440, 2757)]
